In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
from random import randint
num = randint(1, 10)

In [3]:
import io
def get_curve(txt):
    out = io.BytesIO(txt)
    out.seek(0)
    return np.load(out)

In [4]:
# cesta k .db suboru - zmenit na spravnu cestu
FILEPATH = '/home/jovyan/data/lightning/data-upjs/detached.db'
# vytvorenie connection na subor .db v sqllite formate
conn = sqlite3.connect(FILEPATH)

In [5]:
# nacitanie ake su tabulky v databaze (to len aby bolo jasne ako sa to zistit - realne su dve - parameters a curves)
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor = conn.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('parameters',), ('curves',), ('auxiliary',)]


In [6]:
df_parameters = pd.read_sql_query("SELECT id, overcontact FROM parameters", conn)

In [7]:
df_parameters.head(70)

,id,overcontact
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
65,65,0
66,66,0
67,67,0
68,68,0


In [8]:
df_parameters.shape

(2255950, 2)

In [9]:
for col in df_parameters.columns:
    print(col)

id
overcontact


In [ ]:
df_curves = pd.read_sql_query("SELECT * FROM curves ORDER BY RANDOM() LIMIT 1000", conn)
conn.close()

In [ ]:
df_curves.head()

In [ ]:
my_curve_values = np.frombuffer(df_curves["Bessell_U"][20])
my_curve_values1 = get_curve(df_curves["Bessell_U"][20])
print(my_curve_values)

In [ ]:
plt.plot(my_curve_values[16:])
plt.show()

In [ ]:
row = df_curves.iloc[70]
row_curves = row[1:]
my_curves = row_curves.apply(get_curve) 
print(my_curves)

In [ ]:
for c in my_curves:
    plt.plot(c)
    
plt.show()

In [ ]:
df1 = df_curves[["id","Bessell_U", "Bessell_B", "Bessell_V", "Bessell_R", "Bessell_I", "SLOAN_u", "SLOAN_g" , "SLOAN_r" , "SLOAN_i" , "SLOAN_z" , "Kepler" , "GaiaDR2" , "TESS" ]]
df2 = df_parameters[["id", "overcontact"]]
df_merged = pd.merge(df1,df2,on="id")
df_merged["Bessell_U"] = df_merged["Bessell_U"].apply(get_curve)
df_merged["Bessell_V"] = df_merged["Bessell_V"].apply(get_curve)
df_merged["Bessell_R"] = df_merged["Bessell_R"].apply(get_curve)
df_merged["Bessell_I"] = df_merged["Bessell_I"].apply(get_curve)
df_merged["Bessell_B"] = df_merged["Bessell_B"].apply(get_curve)

df_merged["SLOAN_u"] = df_merged["SLOAN_u"].apply(get_curve)
df_merged["SLOAN_g"] = df_merged["SLOAN_g"].apply(get_curve)
df_merged["SLOAN_r"] = df_merged["SLOAN_r"].apply(get_curve)
df_merged["SLOAN_i"] = df_merged["SLOAN_i"].apply(get_curve)
df_merged["SLOAN_z"] = df_merged["SLOAN_z"].apply(get_curve)

df_merged["Kepler"] = df_merged["Kepler"].apply(get_curve)
df_merged["GaiaDR2"] = df_merged["GaiaDR2"].apply(get_curve)
df_merged["TESS"] = df_merged["TESS"].apply(get_curve)


df_merged.info()

In [ ]:
df_merged.to_pickle("overcontact_0_all_filters_1k.pkl")